Departamento de Computação Científica <br>
Centro de Informática - UFPB<br>
Algebra Linear Computacional<br>
<br>

# Trabalho: Decomposição e Algoritmo QR


#### Resumo

O objetivo desse trabalho é desenvolver os algoritmos para realizar o método de Gram-Schmidt, Gram-Schmidt modificado, Householder e, finalmente a decomposição QR, para solucionar as questões 1 e 2.

Cada método terá sua respectiva função, visando a reutilização de código e organização.

# Introdução

Considere, para todas as funções, A sendo uma matriz quadrada de ordem n. A precisão de ponto flutuante é de 6 casas decimais.

 A decomposição QR é uma decomposição de uma matriz A em um produto <b>A = QR</b> de uma matriz ortogonal Q e uma matriz triangular superior R. <br>
A decomposição QR é usado frequentemente para resolver o problema de mínimos quadrados.


Foi utilizada a biblioteca <b>numpy</b>, já que fornecem um ótimo suporte para a manipulação de matrizes.

### Decomposição QR com Gram-Schmidt

O Processo de Gram–Schmidt é um algoritmo para obter uma base ortogonal (ou ortonormal) a partir de uma base qualquer.

A função <b>gram_schmidt_qr(A)</b> calcula a matriz ortogonal Q, resultado do método de Gram-Schmidt e popula a matriz triangular superior com os elementos gerados pelo produto interno entre a i-ésima coluna de Q e a j-ésima coluna de A.

In [1]:
import numpy as np

In [2]:
def round_matrix(M, digits): # função para realizar a correção de ponto flutuante das matrizes
    m, n = M.shape
    
    for i in range(m):
        for j in range(n):
            M[i][j] = round(M[i][j], digits)
    

In [3]:

def gram_schmidt_qr(A): 
    
    det = round(np.linalg.det(np.dot(A, A)), 10) #verificação para matrizes LD
    if abs(det) == 0: 
        print("A matriz não forma uma base")
        return None
    
    m, n = A.shape
    Q = np.eye(n, dtype=float) #gera matriz identidade com shape de n
    R = np.zeros((n, n), dtype=float)
    for i in range(n): #subtrai a projeção e calcula o produto interno
        v = A[:, i]
        for j in range(i):
            R[j, i] = np.dot(Q[:, j], A[:, i]) 
            v = v - R[j, i] * Q[:, j]
        R[i, i] = np.linalg.norm(v) #normalização da coluna
        Q[:, i] = v / R[i, i]
    
    round_matrix(Q, 6)
    round_matrix(R, 6)

    return Q, R

### Decomposição QR com Gram-Schmidt Modificado

O método de Gram-Schmidt modificado, proporciona uma melhor precisão numérica no caso de matrizes de grande porte. 

In [4]:
def gram_schmidt_qr_modificado(A): 

    det = round(np.linalg.det(np.dot(A, A)), 10) #verifica se a matriz de entrada é LD
    
    if abs(det) == 0: # se a entrada não for uma matriz LI, retorna
        print("A matriz não forma uma base")
        return None
    
    m, n = A.shape
    Q = np.zeros_like(A, dtype= float)
    R = np.zeros((n, n), dtype= float)
    for i in range(n): 
        v = A[:, i]
        for j in range(i):
            R[j, i] = np.dot(Q[:, j], A[:, i])
            v = v - R[j, i] * Q[:, j]
        R[i, i] = np.linalg.norm(v) #popula a diagonal principal de R
        Q[:, i] = v / R[i, i]

    round_matrix(Q, 6)
    round_matrix(R, 6)

    return Q, R

### Método de Householder

As transformações de Householder são amplamente utilizadas na álgebra linear numérica, para realizar decomposiçes QR e são o primeiro passo do algoritmo QR. Elas também são amplamente utilizadas para a tridiagonalização de matrizes simétricas e para a transformação de matrizes não-simétricas para a forma de Hessenberg.
<br>
Desse modo, esse método irá decompor a matriz A em uma ortogonal Q e uma triangular superior R - como os métodos que usam Gram-Schmidt. Porém este método proporciona uma maior estabilidade numérica e, portanto, uma precisão. 


In [5]:
def householder_qr(A):
    m, n = A.shape
    Q = np.eye(m, dtype = float)
    R = A.copy()
    
    for i in range(m -1):
        x = R[i:, i] # pega a primeira coluna para cada sub-matriz
        e = np.zeros_like(x)
        e[0] = np.linalg.norm(x)
        u = x - e
        u = u / np.linalg.norm(u)
        
        Q_i = np.identity(m - i) - 2 * (np.outer(u, u)) #calculo da reflexão de householder
        Q_n = np.identity(m)

        Q_n[i:, i:] = Q_i # calcula Q_n

        R = np.dot(Q_n, R)
        Q = np.dot(Q, Q_n)
        
    round_matrix(Q, 10)
    round_matrix(R, 10)

    return Q, R

### Algoritmo QR

A fatoração QR decompõe uma matriz A em uma matriz Q com colunas ortonormais e uma matriz R triangular superior. A deve ter colunas linearmente independentes. Caso A seja quadrada, então <b>Q^−1=Q^T</b>.
<br>
<br>
Esse tipo de fatoração é especialmente útil para resolver o problema dos mínimos quadrados, assim como o algoritmo de SVD. O método das equações normais, da forma B=ATA, que usa a fatoração de Cholesky, pode ter κ(A)2. A intenção do algoritmo QR, então, é minimizar esse erro, obtendo κ(A).


In [6]:

def algoritmo_qr(A, k):
    '''
    para construir a matriz Ak = RkQk
    '''

    n = A.shape[0]
    for i in range(k):
        Q, R = householder_qr(A)
        A = np.dot(R, Q)
    
    for i in range(n):
        for j in range(n):
            A[i][j] = round(A[i][j], 6)
    return A


# Resoluções

#### Questão 1

In [7]:
# inicializando as matrizes necessárias
A = np.array([[4, 0], [3, 1]], dtype= float)

In [8]:
B = np.array([[1, 2], [1, 1]], dtype= float)

In [9]:
C = np.array([[4, 8, 1], [0, 2, -2], [3, 6, 7]], dtype= float)

In [10]:
D = np.array([[12, 10, 4], [10, 8, -5], [4, -5, 3]], dtype= float)

In [11]:
E = np.array([[2, -1, -1], [-1, 2, -1], [-1, -1, 2]], dtype= float)

In [12]:
F = np.array([[1, 1, 1], [1, 1, 0], [1, 0, 1]], dtype= float)

In [13]:
G = np.array([[4.75, 2.25, -0.25], [2.25, 4.75, 1.25], [-0.25, 1.25, 4.75]], dtype= float)

In [14]:
H = np.array([[4, -1, -1, 0], [-1, 4, 0, -1], [-1, 0, 4, -1], [0, -1, -1, 4]], dtype= float)

#### Letra A

Será o algoritmo de Gram-Schmidt aplicado as matrizes

In [15]:
gram_schmidt_qr(A)

(array([[ 0.8, -0.6],
        [ 0.6,  0.8]]),
 array([[5. , 0.6],
        [0. , 0.8]]))

In [16]:
gram_schmidt_qr(B)

(array([[ 0.707107,  0.707107],
        [ 0.707107, -0.707107]]),
 array([[1.414214, 2.12132 ],
        [0.      , 0.707107]]))

In [17]:
gram_schmidt_qr(C)

(array([[ 0.8,  0. , -0.6],
        [ 0. ,  1. ,  0. ],
        [ 0.6,  0. ,  0.8]]),
 array([[ 5., 10.,  5.],
        [ 0.,  2., -2.],
        [ 0.,  0.,  5.]]))

In [18]:
gram_schmidt_qr(D)

(array([[ 0.744208,  0.210906,  0.633776],
        [ 0.620174,  0.134213, -0.772898],
        [ 0.248069, -0.968249,  0.030916]]),
 array([[16.124515, 11.163126,  0.620174],
        [ 0.      ,  8.024002, -2.732187],
        [ 0.      ,  0.      ,  6.492345]]))

In [19]:
gram_schmidt_qr(E)

A matriz não forma uma base


In [20]:
gram_schmidt_qr(F)

(array([[ 0.57735 ,  0.408248,  0.707107],
        [ 0.57735 ,  0.408248, -0.707107],
        [ 0.57735 , -0.816497, -0.      ]]),
 array([[ 1.732051,  1.154701,  1.154701],
        [ 0.      ,  0.816497, -0.408248],
        [ 0.      ,  0.      ,  0.707107]]))

In [21]:
gram_schmidt_qr(G)

(array([[ 0.902717, -0.375774,  0.209513],
        [ 0.427603,  0.837403, -0.340459],
        [-0.047511,  0.396926,  0.91662 ]]),
 array([[5.261891, 4.002838, 0.083145],
        [0.      , 3.628331, 3.026098],
        [0.      , 0.      , 3.875993]]))

In [22]:
gram_schmidt_qr(H)

(array([[ 0.942809,  0.204647,  0.228012,  0.131306],
        [-0.235702,  0.935529, -0.016287,  0.262613],
        [-0.235702, -0.116941,  0.928334,  0.262613],
        [ 0.      , -0.263117, -0.293158,  0.919145]]),
 array([[ 4.242641, -1.885618, -1.885618,  0.471405],
        [ 0.      ,  3.800585, -0.409294, -1.871057],
        [ 0.      ,  0.      ,  3.778482, -2.084679],
        [ 0.      ,  0.      ,  0.      ,  3.151354]]))

#### Letra B

Será o algoritmo de Gram-Schmidt Modificado aplicado as matrizes

In [23]:
gram_schmidt_qr_modificado(A)

(array([[ 0.8, -0.6],
        [ 0.6,  0.8]]),
 array([[5. , 0.6],
        [0. , 0.8]]))

In [24]:
gram_schmidt_qr_modificado(B)

(array([[ 0.707107,  0.707107],
        [ 0.707107, -0.707107]]),
 array([[1.414214, 2.12132 ],
        [0.      , 0.707107]]))

In [25]:
gram_schmidt_qr_modificado(C)

(array([[ 0.8,  0. , -0.6],
        [ 0. ,  1. ,  0. ],
        [ 0.6,  0. ,  0.8]]),
 array([[ 5., 10.,  5.],
        [ 0.,  2., -2.],
        [ 0.,  0.,  5.]]))

In [26]:
gram_schmidt_qr_modificado(D)

(array([[ 0.744208,  0.210906,  0.633776],
        [ 0.620174,  0.134213, -0.772898],
        [ 0.248069, -0.968249,  0.030916]]),
 array([[16.124515, 11.163126,  0.620174],
        [ 0.      ,  8.024002, -2.732187],
        [ 0.      ,  0.      ,  6.492345]]))

In [27]:
gram_schmidt_qr_modificado(E)

A matriz não forma uma base


In [28]:
gram_schmidt_qr_modificado(F)

(array([[ 0.57735 ,  0.408248,  0.707107],
        [ 0.57735 ,  0.408248, -0.707107],
        [ 0.57735 , -0.816497, -0.      ]]),
 array([[ 1.732051,  1.154701,  1.154701],
        [ 0.      ,  0.816497, -0.408248],
        [ 0.      ,  0.      ,  0.707107]]))

In [29]:
gram_schmidt_qr_modificado(G)

(array([[ 0.902717, -0.375774,  0.209513],
        [ 0.427603,  0.837403, -0.340459],
        [-0.047511,  0.396926,  0.91662 ]]),
 array([[5.261891, 4.002838, 0.083145],
        [0.      , 3.628331, 3.026098],
        [0.      , 0.      , 3.875993]]))

In [30]:
gram_schmidt_qr_modificado(H)

(array([[ 0.942809,  0.204647,  0.228012,  0.131306],
        [-0.235702,  0.935529, -0.016287,  0.262613],
        [-0.235702, -0.116941,  0.928334,  0.262613],
        [ 0.      , -0.263117, -0.293158,  0.919145]]),
 array([[ 4.242641, -1.885618, -1.885618,  0.471405],
        [ 0.      ,  3.800585, -0.409294, -1.871057],
        [ 0.      ,  0.      ,  3.778482, -2.084679],
        [ 0.      ,  0.      ,  0.      ,  3.151354]]))

#### Letra C

Será o algoritmo de Holseholder aplicado as matrizes

In [31]:
householder_qr(A)

(array([[ 0.8,  0.6],
        [ 0.6, -0.8]]),
 array([[ 5. ,  0.6],
        [ 0. , -0.8]]))

In [32]:
householder_qr(B)

(array([[ 0.70710678,  0.70710678],
        [ 0.70710678, -0.70710678]]),
 array([[1.41421356, 2.12132034],
        [0.        , 0.70710678]]))

In [33]:
householder_qr(C)

(array([[ 0.8,  0. , -0.6],
        [ 0. ,  1. ,  0. ],
        [ 0.6,  0. ,  0.8]]),
 array([[ 5., 10.,  5.],
        [ 0.,  2., -2.],
        [-0., -0.,  5.]]))

In [34]:
householder_qr(D)

(array([[ 0.74420841,  0.21090568, -0.63377649],
        [ 0.62017367,  0.13421271,  0.77289816],
        [ 0.24806947, -0.9682488 , -0.03091593]]),
 array([[16.1245155 , 11.16312611,  0.62017367],
        [-0.        ,  8.02400245, -2.73218722],
        [ 0.        , -0.        , -6.49234454]]))

In [35]:
householder_qr(E)

(array([[ 0.81649658, -0.        ,  0.57735027],
        [-0.40824829,  0.70710678,  0.57735027],
        [-0.40824829, -0.70710678,  0.57735027]]),
 array([[ 2.44948974, -1.22474487, -1.22474487],
        [-0.        ,  2.12132034, -2.12132034],
        [-0.        , -0.        ,  0.        ]]))

In [36]:
householder_qr(F)

(array([[ 0.57735027,  0.40824829, -0.70710678],
        [ 0.57735027,  0.40824829,  0.70710678],
        [ 0.57735027, -0.81649658,  0.        ]]),
 array([[ 1.73205081,  1.15470054,  1.15470054],
        [-0.        ,  0.81649658, -0.40824829],
        [ 0.        ,  0.        , -0.70710678]]))

In [37]:
householder_qr(G)

(array([[ 0.90271724, -0.37577365,  0.20951312],
        [ 0.4276029 ,  0.83740286, -0.34045882],
        [-0.04751143,  0.39692647,  0.9166199 ]]),
 array([[ 5.26189129,  4.0028383 ,  0.08314501],
        [ 0.        ,  3.62833096,  3.0260977 ],
        [-0.        ,  0.        ,  3.87599273]]))

In [38]:
householder_qr(H)

(array([[ 0.94280904,  0.20464687,  0.22801182, -0.13130643],
        [-0.23570226,  0.93552855, -0.01628656, -0.26261287],
        [-0.23570226, -0.11694107,  0.92833384, -0.26261287],
        [ 0.        , -0.26311741, -0.29315805, -0.91914503]]),
 array([[ 4.24264069, -1.88561808, -1.88561808,  0.47140452],
        [-0.        ,  3.80058475, -0.40929374, -1.87105711],
        [-0.        ,  0.        ,  3.77848158, -2.08467949],
        [ 0.        ,  0.        ,  0.        , -3.15135439]]))

# Questão 2

In [39]:
# inicialiando novas matrizes

A = np.array([[2, -1, 0], [-1, 2, -1], [0, -1, 2]], dtype= float)
B = np.array([[3, 1, 0], [1, 4, 2], [0, 2, 1]], dtype= float)
C = np.array([[4, -1, 0], [-1, 3, -1], [0, -1, 2]], dtype= float) 
D = np.array([[1, 1, 0, 0], [1, 2, -1, 0], [0, -1, 3, 1], [0, 0, 1, 4]], dtype= float)
E = np.array([[-2, 1, 0, 0], [1, -3, -1, 0], [0, -1, 1, 1], [0, 0, 1, 3]], dtype= float)

Será aplicado o Algoritmo QR as matrizes

In [40]:
algoritmo_qr(A, 3)

array([[ 3.308411, -0.372193,  0.      ],
       [-0.372193,  2.103947, -0.052177],
       [ 0.      , -0.052177,  0.587642]])

In [41]:
algoritmo_qr(B, 3)

array([[ 5.079670e+00,  7.930600e-01,  0.000000e+00],
       [ 7.930600e-01,  2.989038e+00, -1.600000e-05],
       [ 0.000000e+00, -1.600000e-05, -6.870800e-02]])

In [42]:
algoritmo_qr(C, 3)

array([[ 4.673724, -0.312694, -0.      ],
       [-0.312694,  3.052961, -0.097082],
       [ 0.      , -0.097082,  1.273316]])

In [43]:
algoritmo_qr(D, 3)

array([[ 3.559441,  0.874042,  0.      ,  0.      ],
       [ 0.874042,  3.656601, -0.992478, -0.      ],
       [ 0.      , -0.992478,  2.528909, -0.025503],
       [ 0.      ,  0.      , -0.025503,  0.25505 ]])

In [44]:
algoritmo_qr(E, 3)

array([[-3.74817 ,  0.28296 , -0.      , -0.      ],
       [ 0.28296 , -0.442379, -2.038983, -0.      ],
       [ 0.      , -2.038983,  2.361207, -0.093222],
       [ 0.      ,  0.      , -0.093222,  0.829342]])